In [1]:
import pandas as pd                                # panel data, for handling dataframes
pd.set_option('display.max_columns', None)         # show all columns of the dataframe

import numpy as np                                 # numerical python, linear algebra library

import pylab as plt                                # plotting library
import seaborn as sns                              # plotting library
sns.set(style='white')                             # seaborn style


from sklearn.linear_model import LogisticRegression            # clasificar, logistic regression model   

from sklearn.preprocessing import StandardScaler               # standarized
from sklearn.preprocessing import LabelEncoder               

from sklearn.model_selection import train_test_split as tts    # split data into train and test sets
from sklearn.ensemble import RandomForestRegressor as RFR    # el modelo , el alias es cosa mia


In [2]:
sample = pd.read_csv('../../data/salaries_data.csv')
test = pd.read_csv('../../data/testeo.csv')

In [3]:
del sample['salary']
del sample['salary_currency']

In [4]:
sample.groupby('experience_level')['salary_in_usd'].mean().sort_values(ascending=False)

experience_level
EX    196078.850000
SE    136341.794239
MI     90384.610465
EN     59989.569231
Name: salary_in_usd, dtype: float64

In [5]:
dic = {'EN': 0, 'MI' : 1, 'SE': 2,  'EX' : 3}
sample.experience_level = sample.experience_level.replace(dic)
test.experience_level = test.experience_level.replace(dic)

In [6]:
sample.employment_type.value_counts()

FT    484
PT      8
CT      4
FL      4
Name: employment_type, dtype: int64

In [7]:
sample.groupby('employment_type')['salary_in_usd'].mean().sort_values(ascending=False)

employment_type
CT    222750.000000
FT    113914.630165
FL     48000.000000
PT     35053.375000
Name: salary_in_usd, dtype: float64

In [8]:
sample.employment_type = sample.employment_type.replace({'CT': 3, 'FL': 1 , 'FT':2 , 'PT': 0})
test.employment_type = test.employment_type.replace({'CT': 3, 'FL': 1 , 'FT':2 , 'PT': 0})

In [9]:
sample.company_size.replace({'L':2,'M':1,'S':0},inplace=True)
test.company_size.replace({'L':2,'M':1,'S':0},inplace=True)

In [10]:
job = []
for e in sample.job_title:
    if 'scien' in e.lower():
        job.append('Data Scientist')
    elif 'machine learning' in e.lower() or 'ml' in e.lower():
        job.append('Data Scientist')
    elif 'engineer' in e.lower():
        job.append('Data Engineer')
    elif 'analyst' in e.lower():
        job.append('Data Analyst')
    else : job.append('Other')

sample.job_title = job

In [11]:
job = []
for e in test.job_title:
    if 'scien' in e.lower():
        job.append('Data Scientist')
    elif 'engineer' in e.lower():
        job.append('Data Engineer')
    elif 'analyst' in e.lower():
        job.append('Data Analyst')
    elif 'machine learning' in e.lower() or 'ml' in e.lower():
        job.append('Data Scientist')
    else : job.append('Other')

test.job_title = job

In [12]:
sample = pd.get_dummies(sample, columns = ['job_title'], drop_first=True)
test = pd.get_dummies(test, columns = ['job_title'], drop_first=True)

In [13]:
countries = list(sample.groupby('company_location')['salary_in_usd'].mean().sort_values(ascending=True).index)

d =dict(zip(countries, [e for e in range(len(countries))]))

sample.company_location.replace(d,inplace = True)
test.company_location.replace(d,inplace = True)

In [14]:
av = sum(list(d.values()))/(len(d.values()))
av

21.0

In [15]:
sample['company_location'] = pd.to_numeric(sample['company_location'], errors='coerce').fillna(av)
test['company_location'] = pd.to_numeric(test['company_location'], errors='coerce').fillna(av)


In [16]:
s = sample.columns
t = test.columns
missing = []

for e in s:
    if e not in t:
        missing.append(e)
missing         

['salary_in_usd']

In [17]:
countries = list(sample.groupby('employee_residence')['salary_in_usd'].mean().sort_values(ascending=True).index)

d =dict(zip(countries, [e for e in range(len(countries))]))

sample.employee_residence.replace(d,inplace = True)
test.employee_residence.replace(d,inplace = True)

In [18]:
av = sum(list(d.values()))/(len(d.values()))

In [19]:
sample['employee_residence'] = pd.to_numeric(sample['employee_residence'], errors='coerce').fillna(av)
test['employee_residence'] = pd.to_numeric(test['employee_residence'], errors='coerce').fillna(av)


In [20]:
sample.head()

,work_year,experience_level,employment_type,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_title_Data Engineer,job_title_Data Scientist,job_title_Other
0,2022,2,2,140250,46,100,42,1,1,0,0
1,2022,2,2,135000,46,100,42,1,1,0,0
2,2021,1,2,100000,46,100,42,1,0,0,0
3,2021,1,3,270000,46,100,42,2,0,1,0
4,2021,1,2,26005,17,0,42,2,1,0,0


**ML**

In [20]:
from sklearn.linear_model import LogisticRegression            # clasificar, logistic regression model   

from sklearn.preprocessing import StandardScaler               # standarized
from sklearn.preprocessing import LabelEncoder               

from sklearn.model_selection import train_test_split as tts    # split data into train and test sets
from sklearn.ensemble import RandomForestRegressor as RFR    # el modelo , el alias es cosa mia


In [21]:
X = sample.drop(columns = 'salary_in_usd')
y = sample.salary_in_usd

X_train, X_test, y_train, y_test=tts(X, y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((375, 10), (125, 10), (375,), (125,))

In [22]:

from lazypredict.Supervised import LazyRegressor 

lazy=LazyRegressor()

lazy

In [23]:
lazy.fit(X_train, X_test, y_train, y_test)

100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:01<00:00, 22.62it/s]


(                               Adjusted R-Squared  R-Squared      RMSE  \
 Model                                                                    
 PoissonRegressor                             0.47       0.52  43912.46   
 RANSACRegressor                              0.46       0.51  44399.54   
 Ridge                                        0.45       0.50  44742.69   
 Lasso                                        0.45       0.50  44742.79   
 LassoLarsIC                                  0.45       0.50  44742.84   
 Lars                                         0.45       0.50  44742.84   
 LinearRegression                             0.45       0.50  44742.84   
 TransformedTargetRegressor                   0.45       0.50  44742.84   
 LassoLars                                    0.45       0.50  44742.86   
 RidgeCV                                      0.45       0.50  44752.60   
 BayesianRidge                                0.45       0.50  44764.23   
 SGDRegressor            

In [24]:
#Normalizing
sample.head()

,work_year,experience_level,employment_type,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,job_title_Data Engineer,job_title_Data Scientist,job_title_Other
0,2022,2,2,140250,46,100,42,1,1,0,0
1,2022,2,2,135000,46,100,42,1,1,0,0
2,2021,1,2,100000,46,100,42,1,0,0,0
3,2021,1,3,270000,46,100,42,2,0,1,0
4,2021,1,2,26005,17,0,42,2,1,0,0


In [25]:
X = sample.drop(columns = 'salary_in_usd')
y = sample.salary_in_usd

In [26]:
data_num=pd.DataFrame(StandardScaler().fit_transform(X.iloc[:,:6]),
                      
                      columns=X.iloc[:,:6].columns)


X_norm=pd.concat([data_num, X.iloc[:,6:]], axis=1)

X_norm.head()

,work_year,experience_level,employment_type,employee_residence,remote_ratio,company_location,company_size,job_title_Data Engineer,job_title_Data Scientist,job_title_Other
0,0.86,0.74,0.11,0.71,0.70,0.65,1,1,0,0
1,0.86,0.74,0.11,0.71,0.70,0.65,1,1,0,0
2,-0.61,-0.57,0.11,0.71,0.70,0.65,1,0,0,0
3,-0.61,-0.57,3.67,0.71,0.70,0.65,2,0,1,0
4,-0.61,-0.57,0.11,-1.63,-1.78,0.65,2,1,0,0


In [27]:
X_train, X_test, y_train, y_test=tts(X, y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((375, 10), (125, 10), (375,), (125,))

In [28]:
lazy.fit(X_train, X_test, y_train, y_test)

'tuple' object has no attribute '__name__'
Invalid Regressor(s)


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [00:02<00:00, 15.60it/s]


(                               Adjusted R-Squared  R-Squared      RMSE  \
 Model                                                                    
 PoissonRegressor                             0.47       0.52  43912.46   
 RANSACRegressor                              0.46       0.51  44399.54   
 Ridge                                        0.45       0.50  44742.69   
 Lasso                                        0.45       0.50  44742.79   
 LassoLarsIC                                  0.45       0.50  44742.84   
 Lars                                         0.45       0.50  44742.84   
 LinearRegression                             0.45       0.50  44742.84   
 TransformedTargetRegressor                   0.45       0.50  44742.84   
 LassoLars                                    0.45       0.50  44742.86   
 RidgeCV                                      0.45       0.50  44752.60   
 BayesianRidge                                0.45       0.50  44764.23   
 SGDRegressor            

In [29]:
from sklearn.linear_model import PoissonRegressor
from sklearn.model_selection import train_test_split

In [30]:
# normalizing test
test.iloc[:,:]

,work_year,experience_level,employment_type,employee_residence,remote_ratio,company_location,company_size,job_title_Data Engineer,job_title_Data Scientist,job_title_Other
0,2020,2,2,37.00,0,34.00,0,0,1,0
1,2020,1,2,46.00,100,42.00,2,0,0,0
2,2020,1,2,46.00,100,42.00,2,0,0,0
3,2020,1,2,24.00,0,38.00,1,1,0,0
4,2020,1,2,15.00,100,13.00,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...
102,2022,2,2,46.00,100,42.00,1,0,0,0
103,2022,2,2,46.00,100,42.00,1,0,0,1
104,2022,2,2,46.00,100,42.00,1,0,1,0
105,2022,0,2,40.00,0,40.00,1,0,0,0


In [31]:
data_num=pd.DataFrame(StandardScaler().fit_transform(test.iloc[:,:7]),
                      
                      columns=test.iloc[:,:7].columns)


test_norm=pd.concat([data_num, test.iloc[:,7:]], axis=1)

test_norm.head()

,work_year,experience_level,employment_type,employee_residence,remote_ratio,company_location,company_size,job_title_Data Engineer,job_title_Data Scientist,job_title_Other
0,-1.87,0.89,0.10,0.01,-1.61,-0.06,-1.59,0,1,0
1,-1.87,-0.29,0.10,0.80,0.80,0.71,1.27,0,0,0
2,-1.87,-0.29,0.10,0.80,0.80,0.71,1.27,0,0,0
3,-1.87,-0.29,0.10,-1.13,-1.61,0.33,-0.16,1,0,0
4,-1.87,-0.29,0.10,-1.92,0.80,-2.06,1.27,0,0,0


In [32]:
X_norm

,work_year,experience_level,employment_type,employee_residence,remote_ratio,company_location,company_size,job_title_Data Engineer,job_title_Data Scientist,job_title_Other
0,0.86,0.74,0.11,0.71,0.70,0.65,1,1,0,0
1,0.86,0.74,0.11,0.71,0.70,0.65,1,1,0,0
2,-0.61,-0.57,0.11,0.71,0.70,0.65,1,0,0,0
3,-0.61,-0.57,3.67,0.71,0.70,0.65,2,0,1,0
4,-0.61,-0.57,0.11,-1.63,-1.78,0.65,2,1,0,0
...,...,...,...,...,...,...,...,...,...,...
495,0.86,0.74,0.11,0.71,0.70,0.65,1,1,0,0
496,-0.61,-1.88,0.11,-1.79,-1.78,-1.09,2,1,0,0
497,0.86,0.74,0.11,0.23,0.70,0.46,1,0,0,1
498,-0.61,-0.57,0.11,-0.42,0.70,-0.51,2,1,0,0


In [33]:
X_train, X_test, y_train, y_test=tts(X_norm, y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((375, 10), (125, 10), (375,), (125,))

In [34]:
pr = PoissonRegressor()
pr.fit(X_train, y_train)

PoissonRegressor()

In [35]:
y_pred = pr.predict(X_test)
mae = np.mean(np.abs(y_test - y_pred))

print("Mean absolute error:", mae)

Mean absolute error: 31113.390476280496


In [36]:
from sklearn.linear_model import HuberRegressor, LinearRegression
from sklearn.datasets import make_regression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error as msle
from sklearn.metrics import mean_absolute_error as mae  # estos alias tmb son cosa mia
from sklearn.metrics import r2_score as r2

r_2 = r2(y_test, y_pred)

print("R squared error:", r_2)

R squared error: 0.5161834411326457


In [37]:
X_train, X_test, y_train, y_test=tts(X_norm, y, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape
pr = PoissonRegressor()
pr.fit(X_train, y_train)
y_pred = pr.predict(test_norm)


In [38]:
result_v2 = pd.DataFrame(y_pred)

In [39]:
result_v2.columns = ['salary_in_usd']

In [40]:
result_v2.to_csv('../data/result_v4.csv')